In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

from qubic.lib.Qgps import GPSCalsource, GPSAntenna, GPStools
 
%matplotlib inline

In [ ]:
data_path = "calsource_orientation_20240322.dat"

In [ ]:
distance_between_antennas = 1
distance_calsource = 0.5
gps_antenna = GPSAntenna(data_path, distance_between_antennas)

In [ ]:
gps_antenna.plot_gps_data()

In [ ]:
index=9615
print(gps_antenna._datetime[index])
print(len(gps_antenna._datetime))

In [ ]:
position_ini_antenna1 = gps_antenna.position_antenna1[:, index]
position_ini_antenna2 = gps_antenna.position_antenna2[:, index]
position_ini_calsource = ((position_ini_antenna2 + position_ini_antenna1)/2)
#position_ini_calsource = np.array([0.5, 2, 0.4])
observation_date = np.array([dt.datetime(2024, 3, 22, 10, 8, 00)]) # np.array([dt.datetime(2024, 3, 22, 9, 47, 49), dt.datetime(2024, 3, 22, 16, 13, 49, 500000)])#
gps = GPSCalsource(data_path, position_ini_antenna1, position_ini_antenna2, position_ini_calsource, distance_between_antennas, observation_date, position_qubic = np.array([0, 0, 0]), ini_wrt_antenna2=False)

In [ ]:
%matplotlib widget
gps.plot_system(index)

In [ ]:
print('Calsource position', gps.position_calsource[:, index])
print('Calsource position ini', gps.position_ini_calsource)
print('Angles', gps.calsource_orientation_angles[:, index])
print('Angles_ini', gps.calsource_orientation_angles_ini[:, 0])

In [ ]:
index_bis = index + 10000

In [ ]:
gps.plot_calsource_deviation(index_bis)

In [ ]:
gps.vector_calsource_deviated

In [ ]:
angles = gps.get_angles(gps.vector_1_2[:, index_bis], gps.vector_1_2_ini)
print(angles)

In [ ]:
calsource_qubic_vector = - gps.get_calsource_qubic_vector(gps.vector_calsource_qubic_ini, angles)

In [ ]:
### Initialize the 3d figure
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(gps.position_ini_calsource[0], gps.position_ini_calsource[1], gps.position_ini_calsource[2], color='red', label='Calsource initial position')
ax.scatter(gps.position_calsource[0, index_bis], gps.position_calsource[1, index_bis], gps.position_calsource[2, index_bis], color='green', label='Calsource position')
ax.scatter(gps.position_qubic[0], gps.position_qubic[1], gps.position_qubic[2], color='blue', label='Qubic position')
ax.scatter(gps.position_ini_antenna1[0], gps.position_ini_antenna1[1], gps.position_ini_antenna1[2], label='Antenna 1 initial position')
ax.scatter(gps.position_antenna1[0, index_bis], gps.position_antenna1[1, index_bis], gps.position_antenna1[2, index_bis], label='Antenna 1 position')
ax.scatter(gps.position_ini_antenna2[0], gps.position_ini_antenna2[1], gps.position_ini_antenna2[2], label='Antenna 2 initial position')
ax.scatter(gps.position_antenna2[0, index_bis], gps.position_antenna2[1, index_bis], gps.position_antenna2[2, index_bis], label='Antenna 2 position')

ax.quiver(gps.position_ini_antenna1[0], gps.position_ini_antenna1[1], gps.position_ini_antenna1[2],
          gps.vector_1_2_ini[0], gps.vector_1_2_ini[1], gps.vector_1_2_ini[2])
ax.quiver(gps.position_antenna1[0, index_bis], gps.position_antenna1[1, index_bis], gps.position_antenna1[2, index_bis],
          gps.vector_1_2[0, index_bis], gps.vector_1_2[1, index_bis], gps.vector_1_2[2, index_bis])

ax.quiver(gps.position_ini_calsource[0], gps.position_ini_calsource[1], gps.position_ini_calsource[2],
          -gps.vector_calsource_qubic_ini[0], -gps.vector_calsource_qubic_ini[1], -gps.vector_calsource_qubic_ini[2], 
          color='red', label='Calsource vector inittial', arrow_length_ratio=0.1)
ax.quiver(gps.position_calsource[0, index_bis], gps.position_calsource[1, index_bis], gps.position_calsource[2, index_bis],
          -calsource_qubic_vector[0], -calsource_qubic_vector[1], -calsource_qubic_vector[2], 
          color='green', label='Calsource vector', arrow_length_ratio=0.1)

plt.legend()
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_zlim(-1, 1)